# ML4UVA

---

## Jupyter Setup

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import sklearn
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

import sys, os, math

from pandas.core.interchange import dataframe

%matplotlib inline

RANDOM_SEED = 42
sklearn.random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
keras.utils.set_random_seed(RANDOM_SEED)

In [2]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

---

## Data Cleaning
### Import Flight Data
Imports and summarizes the dataset

In [3]:
def load_flight_data():
    flight_data = []
    for year in range(2009, 2010):
        flight_data.append(pd.read_csv('kaggle_flight_delay_' + str(year) + '.csv'))

    return pd.concat(flight_data)


data = load_flight_data()

# SIZE (MB)
print('The training set uses', '%.1f' % (data.memory_usage(deep=True).sum() / (1024 ** 3)), '\bGB of memory.')

# ENTRIES
print('The training set has', data.shape[0], 'entries.')

# FEATURES
print('The training set has', data.shape[1], 'features.')

# CATEGORICAL DATA
print('The training set', 'has' if not data.select_dtypes(include=['category', 'object']).empty else 'does not have',
      'categorical data.')

# MISSING DATA
print('The training set', 'has' if data.isnull().any(axis=None) else 'does not have', 'missing data.', end='\n\n')

# BASIC STATISTICS
data.info()
data.describe()

The training set uses 2.8 GB of memory.
The training set has 6429338 entries.
The training set has 28 features.
The training set has categorical data.
The training set has missing data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6429338 entries, 0 to 6429337
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   OP_CARRIER_FL_NUM    int64  
 3   ORIGIN               object 
 4   DEST                 object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  CANCELLED            float64
 16  CANCELLATION_CODE    object 
 17  DIVERTED             float64
 18  CRS_ELAPSED_TI

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
count,6.429338e+06,6.429338e+06,6.346471e+06,6.346471e+06,6.343551e+06,6.343551e+06,6.340016e+06,6.340017e+06,6.429338e+06,6.340016e+06,...,6.429338e+06,6.326976e+06,6.326977e+06,6.429338e+06,1.170501e+06,1.170501e+06,1.170501e+06,1.170501e+06,1.170501e+06,0.0
mean,2.293712e+03,1.319447e+03,1.326026e+03,7.598179e+00,1.603613e+01,1.350798e+03,1.478506e+03,6.894225e+00,1.496722e+03,1.485662e+03,...,1.293446e+02,1.262434e+02,1.033257e+02,7.249700e+02,1.546616e+01,2.755585e+00,1.644337e+01,6.408282e-02,1.953755e+01,NaN
std,2.046497e+03,4.577529e+02,4.688940e+02,3.176494e+01,1.056383e+01,4.700515e+02,4.878461e+02,4.908826e+00,4.733178e+02,4.904066e+02,...,6.966364e+01,6.950774e+01,6.706977e+01,5.610229e+02,3.998500e+01,1.790554e+01,2.979917e+01,1.745892e+00,3.705886e+01,NaN
min,1.000000e+00,1.000000e+00,1.000000e+00,-9.600000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.400000e+01,0.000000e+00,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25%,6.180000e+02,9.260000e+02,9.280000e+02,-5.000000e+00,1.000000e+01,9.440000e+02,1.107000e+03,4.000000e+00,1.119000e+03,1.112000e+03,...,8.000000e+01,7.600000e+01,5.500000e+01,3.250000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
50%,1.629000e+03,1.315000e+03,1.322000e+03,-2.000000e+00,1.300000e+01,1.336000e+03,1.510000e+03,6.000000e+00,1.519000e+03,1.515000e+03,...,1.110000e+02,1.090000e+02,8.500000e+01,5.810000e+02,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00,NaN
75%,3.536000e+03,1.710000e+03,1.719000e+03,5.000000e+00,1.900000e+01,1.734000e+03,1.857000e+03,8.000000e+00,1.903000e+03,1.903000e+03,...,1.590000e+02,1.550000e+02,1.310000e+02,9.480000e+02,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.500000e+01,NaN
max,7.829000e+03,2.359000e+03,2.400000e+03,2.445000e+03,4.580000e+02,2.400000e+03,2.400000e+03,1.970000e+02,2.400000e+03,2.400000e+03,...,6.600000e+02,7.990000e+02,6.990000e+02,4.962000e+03,2.439000e+03,1.114000e+03,1.305000e+03,3.450000e+02,1.391000e+03,NaN


### Data Pre-Processing
Sets up two data pre-processors: the first coverts datatypes and correct erroneous values, and the second drops duplicate and unused data columns.

In [4]:
from sklearn.preprocessing import FunctionTransformer

def convert_datatype(dataframe):
    dataframe = dataframe.astype({
        'CANCELLATION_CODE': str,

        # DURATION (NULLABLE)
        'DEP_DELAY': 'Int64',
        'ARR_DELAY': 'Int64',
        'TAXI_OUT': 'Int64',
        'TAXI_IN': 'Int64',
        'CRS_ELAPSED_TIME': 'Int64',
        'ACTUAL_ELAPSED_TIME': 'Int64',
        'AIR_TIME': 'Int64',
        'CARRIER_DELAY': 'Int64',
        'WEATHER_DELAY': 'Int64',
        'NAS_DELAY': 'Int64',
        'SECURITY_DELAY': 'Int64',
        'LATE_AIRCRAFT_DELAY': 'Int64',

        # TIME (NULLABLE)
        'DEP_TIME': 'Int64',
        'ARR_TIME': 'Int64',
        'WHEELS_OFF': 'Int64',
        'WHEELS_ON': 'Int64',

        # Changed it to Int64 to handle NaN and prevent errors
        'DISTANCE': 'Int64',

        'CANCELLED': bool,
        'DIVERTED': bool,
    })

    # CONVERT TO DAY, MONTH, YEAR
    # Add coerce which cverts any invalid dates formats to NAT and drop those dates.
    dataframe['FLIGHT_DATE'] = pd.to_datetime(dataframe['FL_DATE'], errors ='coerce')
    dataframe = dataframe.dropna(subset=["FLIGHT_DATE"])
    dataframe['DAY'] = dataframe['FLIGHT_DATE'].apply(lambda x: x.day)
    dataframe['MONTH'] = dataframe['FLIGHT_DATE'].apply(lambda x: x.month)
    dataframe['YEAR'] = dataframe['FLIGHT_DATE'].apply(lambda x: x.year)

    '''
    # Converts the time (HHMM) to the number of minutes since midnight, which improves model consistency.
    # Otherwise, 1159 (11:59AM) and 1200 (12:00PM) are treated as 41 minutes apart, which is incorrect.
    # '''
    for column in [
        'CRS_DEP_TIME',
        'DEP_TIME',
        'CRS_ARR_TIME',
        'ARR_TIME',
        'WHEELS_OFF',
        'WHEELS_ON',
    ]:
        dataframe[column] = dataframe[column].apply(lambda x: x % 100 + 60 * (x // 100))

    # CONVERTED FOR CLARITY
    cancellation_codes = {
        'nan': 'NONE',
        'A': 'AIRLINE',
        'B': 'WEATHER',
        'C': 'NAS',
        'D': 'SECURITY'
    }
    dataframe['CANCELLATION_CODE'] = dataframe['CANCELLATION_CODE'].apply(lambda x: cancellation_codes[x])

    airport_region_map = {
        'BOS': 'New England', 'BDL': 'New England', 'PWM': 'New England', 'PVD': 'New England',
        'ALB': 'New England', 'MHT': 'New England', 'BGR': 'New England',

        'JFK': 'Mid-Atlantic', 'LGA': 'Mid-Atlantic', 'EWR': 'Mid-Atlantic', 'PHL': 'Mid-Atlantic',
        'DCA': 'Mid-Atlantic', 'BWI': 'Mid-Atlantic', 'ACY': 'Mid-Atlantic',

        'ORD': 'Midwest', 'MDW': 'Midwest', 'DTW': 'Midwest', 'CLE': 'Midwest',
        'IND': 'Midwest', 'CMH': 'Midwest', 'MKE': 'Midwest', 'GRR': 'Midwest',

        'MSP': 'Great Plains', 'OMA': 'Great Plains', 'DSM': 'Great Plains', 'MCI': 'Great Plains',
        'STL': 'Great Plains', 'BIS': 'Great Plains', 'FSD': 'Great Plains',

        'ATL': 'Southeast', 'CLT': 'Southeast', 'MIA': 'Southeast', 'FLL': 'Southeast', 'TPA': 'Southeast',
        'BNA': 'Southeast', 'RDU': 'Southeast', 'CHS': 'Southeast', 'JAX': 'Southeast', 'MEM': 'Southeast',
        'SAV': 'Southeast', 'MSY': 'Southeast', 'BHM': 'Southeast',

        'DFW': 'Southwest', 'DAL': 'Southwest', 'HOU': 'Southwest', 'IAH': 'Southwest',
        'PHX': 'Southwest', 'AUS': 'Southwest', 'ABQ': 'Southwest', 'SAT': 'Southwest',
        'ELP': 'Southwest', 'LBB': 'Southwest', 'TUL': 'Southwest', 'OKC': 'Southwest',

        'DEN': 'Mountain', 'SLC': 'Mountain', 'COS': 'Mountain', 'BOI': 'Mountain',
        'MSO': 'Mountain', 'BZN': 'Mountain', 'GJT': 'Mountain', 'JAC': 'Mountain',

        'LAX': 'Pacific', 'SFO': 'Pacific', 'SEA': 'Pacific', 'PDX': 'Pacific',
        'SAN': 'Pacific', 'SJC': 'Pacific', 'SMF': 'Pacific', 'ONT': 'Pacific',
        'OGG': 'Pacific', 'KOA': 'Pacific', 'HNL': 'Pacific', 'ANC': 'Pacific',
        'FAI': 'Pacific', 'LIH': 'Pacific'
    }

    # Example application to the DataFrame
    dataframe['DEST'] = dataframe['DEST'].map(airport_region_map)
    dataframe['ORIGIN'] = dataframe['ORIGIN'].map(airport_region_map)


    return dataframe

datatype_converter = FunctionTransformer(convert_datatype)

In [5]:
def extract_features(dataframe):
    # DUPLICATE OR NULL DATA
    dataframe = dataframe.drop(columns=['Unnamed: 27', 'FL_DATE', 'FLIGHT_DATE'])

    # DATA MEASURED AFTER TAKEOFF
    dataframe = dataframe.drop(columns=['DEP_TIME', 'TAXI_OUT', 'TAXI_IN', 'WHEELS_OFF', 'WHEELS_ON', 'ARR_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME'])

    '''
    This data measures the cause of delay, in minutes, of the plane's arrival. While potentially useful, this data DOES NOT explain the reason for the departure delay. Unfortunately, this data is only reported if the aircraft arrives less than fifteen minutes late, significantly reducing its utility. Each of these is described in more detail below (paraphrased from the Bureau of Transportation Statistics):

    - Air Carrier (CARRIER_DELAY): Circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling)
    - Extreme Weather (WEATHER_DELAY): Extreme meteorological conditions that delays or prevents the operation of a flight (e.g. tornado, blizzard or hurricane).
    - National Aviation System (NAS_DELAY): Delays and cancellations attributable to the national aviation system, such as non-extreme weather conditions (like thunderstorms), airport operations, heavy traffic volume, and air traffic control.
    - Late-arriving aircraft (LATE_AIRCRAFT_DELAY): The previous flight arrived late, causing the present flight to depart late.
    - Security (SECURITY_DELAY): Delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.

    In my opinion, this data is not very useful because it is impractical to determine whether weather was the cause of the delay.
    '''
    arrival_delay_data = dataframe[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'ARR_DELAY']]
    dataframe = dataframe.drop(columns=['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'ARR_DELAY'])

    # DIVERSIONS (THESE USUALLY OCCUR AFTER TAKEOFF)
    dataframe = dataframe.drop(columns=['DIVERTED'])

    '''
    Since we have a lot of clean, usable data, I decided to drop any rows with missing flight numbers, origins, destinations, date information, and labels. The other columns can be imputed without losing much data integrity.
    '''
    dataframe = dataframe.dropna(subset=['OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'DEP_DELAY', 'CANCELLED', 'DAY', 'MONTH', 'YEAR'])

    return dataframe

feature_extractor = FunctionTransformer(extract_features)

### Data Pipeline
Uses two preprocessing steps, generates labels, then transforms numeric and categorical data.

In [6]:
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

column_transformer = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, make_column_selector(dtype_include=[np.number])),
        ('categorical', categorical_transformer, make_column_selector(dtype_include=['object', 'category'])),
    ]
)

preprocessor = Pipeline(steps=[
    ('datatype_converter', datatype_converter),
    ('feature_extractor', feature_extractor),
])
transformer = Pipeline(steps=[
    ('transformer', column_transformer),
])

In [7]:
# PREPROCESS
preprocessed_data = preprocessor.fit_transform(data)

# LABELS
cancellation_data = preprocessed_data['CANCELLATION_CODE']
delay_data = preprocessed_data['DEP_DELAY']
preprocessed_data = preprocessed_data.drop(columns=['CANCELLATION_CODE', 'CANCELLED', 'DEP_DELAY'])

# TRANSFORMER
processed_data = transformer.fit_transform(preprocessed_data)

In [8]:
feature_names = column_transformer.get_feature_names_out()

processed_df = pd.DataFrame(processed_data, columns=feature_names)

print("One hot encoder new columns:")
print(processed_df.columns.tolist())

One hot encoder new columns:
['numerical__OP_CARRIER_FL_NUM', 'numerical__CRS_DEP_TIME', 'numerical__CRS_ARR_TIME', 'numerical__CRS_ELAPSED_TIME', 'numerical__DISTANCE', 'numerical__DAY', 'numerical__MONTH', 'numerical__YEAR', 'categorical__OP_CARRIER_9E', 'categorical__OP_CARRIER_AA', 'categorical__OP_CARRIER_AS', 'categorical__OP_CARRIER_B6', 'categorical__OP_CARRIER_CO', 'categorical__OP_CARRIER_DL', 'categorical__OP_CARRIER_EV', 'categorical__OP_CARRIER_F9', 'categorical__OP_CARRIER_FL', 'categorical__OP_CARRIER_HA', 'categorical__OP_CARRIER_MQ', 'categorical__OP_CARRIER_NW', 'categorical__OP_CARRIER_OH', 'categorical__OP_CARRIER_OO', 'categorical__OP_CARRIER_UA', 'categorical__OP_CARRIER_US', 'categorical__OP_CARRIER_WN', 'categorical__OP_CARRIER_XE', 'categorical__OP_CARRIER_YV', 'categorical__ORIGIN_Great Plains', 'categorical__ORIGIN_Mid-Atlantic', 'categorical__ORIGIN_Midwest', 'categorical__ORIGIN_Mountain', 'categorical__ORIGIN_New England', 'categorical__ORIGIN_Pacific', 'c

### Training, Testing, and Validation Datasets & Labels
Creates training labels for the amount of delay in minutes and a boolean delay status with a fifteen-minute threshold. The dataset is split into training, validation, and test datasets.

In [10]:
delay_labels = delay_data.combine(cancellation_data, lambda delay, code: delay if not pd.isna(code) else 120)
delay_statuses = delay_labels > 15

In [12]:
from sklearn.model_selection import train_test_split

full_training, test, full_training_delays, test_delays, full_training_statuses, test_statuses = train_test_split(
    processed_data, delay_labels, delay_statuses, test_size=0.2, random_state=RANDOM_SEED
)
training, validation, training_delays, validation_delays, training_statuses, validations_statuses = train_test_split(
    full_training, full_training_delays, full_training_statuses, test_size=0.2, random_state=RANDOM_SEED
)

---

## Models
### Linear Regression

In [13]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import root_mean_squared_error

linear_regressor = SGDRegressor(random_state=RANDOM_SEED)
linear_regressor.fit(training, training_delays)

linear_delay_predictions = linear_regressor.predict(validation)
linear_rmse = root_mean_squared_error(validation_delays, linear_delay_predictions)

In [14]:
print(linear_rmse)

31.333839070780027
